#### Week 1: Building Advanced RAG Applications.  Authored by Chris Sanchez.

# Week 1 - Notebook 3

# Overview
***
Welcome to the final notebook for Week 1! Take a look at all the ground we've covered so far:
- Chunking/splitting
- Text vectorization
- Combining metadata
- Collection Configuration
- Data Indexing
- Keyword search
- Vector search
- OPTIONAL: Searching with Filters

We are now prepared to move on to a very important topic, **Retrieval Evaluation**.  I hope you've noticed that the search results will differ (sometimes slightly, sometimes by a lot) depeding on which search method you used: `keyword_search` or `vector_search`.  As humans, it's fairly easy for us to determine whether the returned search results are relevant to the query that was submitted, (though even here there will be differing opinions on result relevance).  But how do we systematically determine which search method is better in general?  And how do we measure the relative performance of our retrieval system if we change one of it's parameters...for example, changing our embedding model? What about measuring system performance over time as more documents are added to our datastore?

We need a way to evaluate our retrieval system, and this notebook will show you "one way" of doing that.  I say "one way" because there are many ways to approach this problem, and the method I'm showing you is not perfect (if anything it's a bit too conservative).  Ultimately, measuring retrieval performance is hard because it requires a lot of time and effort, and absent any user [click-data](https://en.wikipedia.org/wiki/Click_tracking), requires some form of data labeling.  With the advent of powerful generative LLMs the process of measuring retrieval performance has become much easier. Let's take a look at how that works.

# Retrieval Evaluation - Process
***
Here's a high-level overview of how the Retrieval Evaluation process in this notebook works:

1. Generate a "golden dataset" of query-context pairs.  I've already pre-generated multiple golden datasets for our course. I randomly selected 100 document chunks (contexts) from the Huberman Lab corpus and those chunks were then submitted to the `gpt-3.5-turbo` model which generated queries that can be answered by the context.  The output was 100 query-context pairs along with their associated doc_ids. 
   - **Baseline Assumptions**:
     - The generated query-context pairs are, in fact, relevant to one another i.e. the query can be answered by the context that it's paired with
     - The generated queries are simliar in style and length to the type of queries that end users would ask
2. The golden dataset consists of three primary keys: `corpus`, `relevant_docs`, and `queries`
     - The `corpus` is the original text context/chunk with it's associated `doc_id`
     - The `queries` are the LLM generated queries, one (or more) for each entry in the `corpus`
     - The `relevant_docs` is a simple lookup table linking the `corpus` docs to the generated `queries`
3. We pass the golden dataset into a retrieval evluation function which does the following:
   - Takes in a `retriever` arg (`WeaviateWCS`) and a few other configuration params
   - Iterates over all queries in the golden dataset and retrieves search results for each query from the Weaviate datastore
   - Extracts all `doc_id` values from the retrieved results
   - Extracts the `doc_id` from the associated `relevant_docs` for each query
   - Checks if the relevant doc_id is in the list of retrieved result doc_ids
   - After all queries are completed a `hit_rate` score and `mrr` score are calculated for the entire golden dataset
   - Writes results to an `eval_results` folder

#### In a Nutshell
Ulitmately, given a golden dataset consisting of queries, relevant docs, and their associated doc_ids, the `retrieval_evaluation` function is checking if the relevant doc_id is found in the list of retrieved results doc_ids, for each query.

#### Problems with this Approach
The problems with this approach are many, I'll cover a few here:
- The **Assumptions** (see section 1 above) about the golden dataset must hold true.  Given that the pairs are generated by `gpt-3.5-turbo`, I think the first assumption will generally be true.  When reviewing the dataset I did find a few questions that were not answerable given the context, but for the most part they were.  The 2nd assumption though, is going to be dependent on your particular search use case.  I think for the purposes of this course, the questions generated are a decent reflection of how someone would query this dataset, and therefore do the job of measuring retriever performance.  But I would always check a real-world query distribution before using an approach like the one presented here.
- This approach is conversative in that there is only "one" right answer.  Either the relevant `doc_id` is in the results list or it isn't.  In reality, there are going to be several documents that could potentially answer the generated query, but we have no way to account for these other relevant documents, unless of course, we want to manually add doc_ids to the golden dataset (and depending on your business case, you may actually want to do that).
- We aren't measuring recall or precision because we aren't classifying other documents as "negatives".  As was just mentioned, the other documents in the results list may or may not be good matches, we just don't know.  Because we don't know, we can't really classify the other documents as "negatives".  So for this approach, we are measuring the ["hit rate"](https://uplimit.com/course/rag-applications/v2/module/retreival-evaluation#corise_clp66zqui003i2a777aldseor) which is simply a count of the number of times that we found a relevant `doc_id` match in the results list and [Mean Reciprocal Rank (MRR)](https://uplimit.com/course/rag-applications/v2/module/retreival-evaluation#corise_clp66zqui003j2a77u8lnrk5b).  We're using MRR instead of other metrics such as Mean Average Precision (MAP) because we are only looking at a [single relevant answer](https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank).  Hit rate is a good enough metric for determining if our retriever is retrieving quality results, and MRR will become more important later on when we add a Reranker to the mix.  

In [1]:
#standard library imports
import sys
sys.path.append('../')

from typing import Any
import time
import os

# utilities
from tqdm import tqdm
from rich import print
from dotenv import load_dotenv, find_dotenv
env = load_dotenv(find_dotenv(), override=True)

# Assignment 1.3
***
#### Instructions:
* Import the `golden_100.json` dataset using the `load_json` method of the FileIO Class
* Instantiate a new WeaviateWCS client (Retriever) and set the `collection_name` of the Collection that you created in Notebook 2
* Evaluate your retriever results using the `retrieval_evaluation` function
* Submit your results in the form of a text file to Uplimit (the function autogenerates a report in the `dir_outpath` directory).

In [28]:
from src.evaluation.retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results, add_params
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.database.database_utils import get_weaviate_client
from src.preprocessor.preprocessing import FileIO

#################
##  START CODE ##
#################

# Load QA dataset
data_path = '../data/golden_datasets/golden_128.json'
golden_dataset = FileIO().load_json(data_path)

# should see 100 queries
print(f'Num queries in Golden Dataset: {len(golden_dataset["queries"])}')

### Instantiate Weaviate client and set Collection name
# read env vars from local .env file
client = get_weaviate_client()
collection_name = 'Huberman_minilm_128'

#################
##  END CODE   ##
#################

Num queries in Golden Dataset: 100

/var/folders/bn/cbq1dkw10_b1lphn2wml9k4m0000gn/T/ipykernel_13886/2733157713.py:19: ResourceWarning: unclosed <ssl.SSLSocket fd=98, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.4.117', 63547), raddr=('34.149.137.116', 443)>
  client = get_weaviate_client()


# Project 1: Retrieval Evaluation

In [31]:
def retrieval_evaluation(dataset: dict, 
                         collection_name: str, 
                         retriever: WeaviateWCS,
                         retrieve_limit: int=5,
                         chunk_size: int=256,
                         query_properties: list[str]=['content'],
                         hnsw_config_keys: list[str]=['maxConnections', 'efConstruction', 'ef'],
                         display_properties: list[str]=['doc_id', 'guest', 'content'],
                         dir_outpath: str='./eval_results',
                         include_miss_info: bool=False,
                         user_def_params: dict[str,Any]=None
                         ) -> dict[str, str|int|float]:
    '''
    Given a dataset and a retriever evaluate the performance of the retriever. Returns a dict of kw and vector
    hit rates and mrr scores. If inlude_miss_info is True, will also return a list of kw and vector responses 
    and their associated queries that did not return a hit, for deeper analysis. Text file with results output
    is automatically saved in the dir_outpath directory.

    Args:
    -----
    dataset: dict
        Dataset to be used for evaluation
    collection_name: str
        Name of Collection on Weaviate host to be used for retrieval
    retriever: WeaviateWCS
        WeaviateWCS object to be used for retrieval 
    retrieve_limit: int=5
        Number of documents to retrieve from Weaviate host
    chunk_size: int=256
        Number of tokens used to chunk text. This value is purely for results 
        recording purposes and does not affect results. 
    display_properties: list[str]=['doc_id', 'content']
        list of properties to be returned from Weaviate host for display in response
    dir_outpath: str='./eval_results'
        Directory path for saving results.  Directory will be created if it does not
        already exist. 
    include_miss_info: bool=False
        Option to include queries and their associated kw and vector response values
        for queries that are "total misses"
    user_def_params : dict=None
        Option for user to pass in a dictionary of user-defined parameters and their values.
    '''

    results_dict = {'n':retrieve_limit, 
                    'Retriever': retriever.model_name_or_path, 
                    'chunk_size': chunk_size,
                    'kw_hit_rate': 0,
                    'kw_mrr': 0,
                    'vector_hit_rate': 0,
                    'vector_mrr': 0,
                    'total_misses': 0,
                    'total_questions':0
                    }
    #add hnsw configs and user defined params (if any)
    # results_dict = add_params(client, collection_name, results_dict, user_def_params, hnsw_config_keys)
    
    start = time.perf_counter()
    miss_info = []
    for query_id, q in tqdm(dataset['queries'].items(), 'Queries'):
        results_dict['total_questions'] += 1
        hit = False
        #make Keyword, Vector, and Hybrid calls to Weaviate host
        try:
            kw_response = retriever.keyword_search(request=q, collection_name=collection_name, query_properties=query_properties,
                                                   limit=retrieve_limit, return_properties=display_properties)
            vector_response = retriever.vector_search(request=q, collection_name=collection_name, 
                                                   limit=retrieve_limit, return_properties=display_properties)
            
            #collect doc_ids and position of doc_ids to check for document matches
            kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response, 1)}
            vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response, 1)}
            
            #extract doc_id for scoring purposes
            doc_id = dataset['relevant_docs'][query_id]
 
            #increment hit_rate counters and mrr scores
            if doc_id in kw_doc_ids:
                results_dict['kw_hit_rate'] += 1
                results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
                hit = True
            if doc_id in vector_doc_ids:
                results_dict['vector_hit_rate'] += 1
                results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]
                hit = True
                
            # if no hits, let's capture that
            if not hit:
                results_dict['total_misses'] += 1
                miss_info.append({'query': q, 'kw_response': kw_response, 'vector_response': vector_response})
        except Exception as e:
            print(e)
            continue
    

    #use raw counts to calculate final scores
    calc_hit_rate_scores(results_dict, search_type=['kw', 'vector'])
    calc_mrr_scores(results_dict, search_type=['kw', 'vector'])
    
    end = time.perf_counter() - start
    print(f'Total Processing Time: {round(end/60, 2)} minutes')
    record_results(results_dict, chunk_size, dir_outpath=dir_outpath, as_text=True)
    
    if include_miss_info:
        return results_dict, miss_info
    return results_dict

### Run evaluation over golden dataset

In [39]:
#################
##  START CODE ##
#################
results = retrieval_evaluation(golden_dataset, collection_name, client, query_properties=['content'], chunk_size=128)

Queries: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:37<00:00,  2.65it/s]


Total Processing Time: 0.63 minutes

In [40]:
results

{'n': 5,
 'Retriever': 'sentence-transformers/all-MiniLM-L6-v2',
 'chunk_size': 128,
 'kw_hit_rate': 0.84,
 'kw_mrr': 0.77,
 'vector_hit_rate': 0.66,
 'vector_mrr': 0.54,
 'total_misses': 9,
 'total_questions': 100}

# Conclusion
***

We now have a way of measuring the performance of our system.  This will allow you to make tweaks/changes to the system and then be able to objectively tell whether or not the tweak/change improved or degraded its performance.  Here are a few things to consider going forward:  

- Keep in mind what the ulitmate goal of the system is that you are building.  For this course, we are trying to retrieve the most relevant documents as possible that will effectively address a user query, assuming the information is found within the corpus.  This means that we don't need pages and pages of relevant results, we actually only need the top 3-5, just enough to allow our Reader (the OpenAI LLM) to answer the user query.  This is an important point to be thinking about as you are making changes to the retrieval system.
- Feel free to set the `include_miss_info` param to `True`.  Doing so will return a list of both the keyword and vector responses that did not contain the relevant `doc_id` (a "total_miss" means the `doc_id` was not present in either the `kw_doc_ids` or the `vector_doc_ids`).  Take a look at the style of the queries being asked and compare them with the returned responses.  Why are those responses being returned?  Are they close to the intent of the query?
- Last but not least, you are now free to make changes to your system to improve the `hit_rate` and `mrr` scores.  If it were me, I'd start with switching out to a more performant [embedding model](https://huggingface.co/spaces/mteb/leaderboard).  There will be more opportunities to pick up some low hanging fruit, but we'll have to wait until the following week when hybrid search and Rerankers are introduced.  Whatever you do though, don't change params for the `SentenceSplitter` that you use for chunking the corpus.  Due to the way the golden dataset is derived, it's unfortunately dependent on those original `SentenceSplitter` settings remaining the same across evaluations. That is, of course, unless you want to build out your own golden dataset....